In [4]:
# pip install yfinance pandas
import yfinance as yf
import pandas as pd

In [ ]:
# 1) Put your tickers here (can be hundreds+). Example:
tickers = ["AAPL","MSFT","GOOGL","AMZN","META","TSLA"]

# 2) Download daily prices (adjusted)
data = yf.download(
    tickers=" ".join(tickers),
    start="2010-01-01",
    auto_adjust=True,      # ensures dividends/splits included in price
    progress=False,
    group_by="ticker",     # returns a column MultiIndex if many tickers
    threads=True
)

# 3) Extract adjusted closes into a clean wide DataFrame
def get_adj_close(df):
    if isinstance(df.columns, pd.MultiIndex):
        # multi-ticker shape: top level = ticker, second level = fields
        closes = pd.concat({t: df[t]["Close"] for t in tickers}, axis=1)
    else:
        # single-ticker shape: columns like ['Open','High',...]
        closes = df["Close"].to_frame(tickers[0])
    return closes

adj_close = get_adj_close(data).sort_index()

# 4) Convert to monthly prices (month-end) and compute simple returns
monthly_prices = adj_close.resample("M").last()
monthly_returns = monthly_prices.pct_change()

# (optional) log returns:
monthly_log_returns = (monthly_prices / monthly_prices.shift(1)).applymap(lambda x: pd.NA if pd.isna(x) else pd.Series([pd.np.log(x)])).iloc[:,0]

# 5) Tidy (long) format if you prefer
monthly_returns_long = (
    monthly_returns
    .stack()
    .rename("ret")
    .reset_index()
    .rename(columns={"level_1": "ticker", "Date": "month"})
)

print(monthly_returns.head())
print(monthly_returns_long.head())